# Paramount+ Streaming Analytics Notebook

This notebook analyzes streaming data for Paramount+ sports content (UFC, UEFA Champions League, NFL) to understand viewer behavior, engagement patterns, and identify key moments.

## Objectives
- Load and analyze viewing logs (who watched what, when, on which device)
- Join with content metadata (league, team, language, live vs replay, tournament stage)
- Build metrics: drop-off analysis, watch time by sport/league/team, big moments detection
- Create moment maps showing where fandom is intense, where people churn, where they binge


In [ ]:
# Install required packages
%pip install pandas plotly nbformat ipywidgets


Note: you may need to restart the kernel to use updated packages.


## Download UFC Dataset from GitHub

This section clones the UFC-DataLab repository from GitHub which contains comprehensive UFC fight data, fighter stats, and official fight scorecards.

Repository: https://github.com/komaksym/UFC-DataLab.git


In [ ]:
# Clone UFC-DataLab repository from GitHub
import subprocess
import os
from pathlib import Path

# Set up paths
repo_name = 'UFC-DataLab'
repo_url = 'https://github.com/komaksym/UFC-DataLab.git'
data_dir = Path('data')
ufc_repo_dir = Path(repo_name)

print(f"Cloning repository: {repo_url}")
print("This may take a few moments...")

# Clone the repository if it doesn't exist
if not ufc_repo_dir.exists():
    try:
        result = subprocess.run(
            ['git', 'clone', repo_url],
            capture_output=True,
            text=True,
            check=True
        )
        print(f"✓ Repository cloned successfully to {ufc_repo_dir}/")
    except subprocess.CalledProcessError as e:
        print(f"⚠️ Error cloning repository: {e.stderr}")
        print("\nMake sure git is installed. You can also:")
        print(f"1. Manually clone: git clone {repo_url}")
        print(f"2. Or download as ZIP from: {repo_url}")
else:
    print(f"✓ Repository already exists at {ufc_repo_dir}/")
    print("Pulling latest changes...")
    try:
        subprocess.run(
            ['git', '-C', str(ufc_repo_dir), 'pull'],
            capture_output=True,
            text=True,
            check=True
        )
        print("✓ Repository updated")
    except subprocess.CalledProcessError:
        print("⚠️ Could not update repository (this is okay if you don't need latest changes)")


In [ ]:
# Explore the UFC-DataLab repository structure
import pandas as pd

if ufc_repo_dir.exists():
    print(f"📂 Repository structure:")
    print(f"   Location: {ufc_repo_dir.absolute()}")
    
    # Check for data directories
    data_paths = {
        'stats': ufc_repo_dir / 'data' / 'stats',
        'scorecards': ufc_repo_dir / 'data' / 'scorecards',
        'merged': ufc_repo_dir / 'data' / 'merged_stats_n_scorecards',
        'external': ufc_repo_dir / 'data' / 'external_data'
    }
    
    print("\n📊 Available data directories:")
    for name, path in data_paths.items():
        if path.exists():
            csv_files = list(path.glob('*.csv'))
            print(f"   ✓ {name}: {path} ({len(csv_files)} CSV files)")
        else:
            print(f"   ✗ {name}: Not found")
    
    # Find all CSV files in the repository
    all_csv_files = list(ufc_repo_dir.rglob('*.csv'))
    print(f"\n📄 Total CSV files found: {len(all_csv_files)}")
    
    if all_csv_files:
        print("\n📋 Sample files:")
        for i, file in enumerate(all_csv_files[:10], 1):
            size_mb = file.stat().st_size / 1024 / 1024
            print(f"   {i}. {file.relative_to(ufc_repo_dir)} ({size_mb:.2f} MB)")
        if len(all_csv_files) > 10:
            print(f"   ... and {len(all_csv_files) - 10} more files")
else:
    print("⚠️ Repository not found. Please run the previous cell to clone it.")


In [ ]:
# Load and preview key UFC datasets
if ufc_repo_dir.exists():
    # Try to load the merged dataset (most comprehensive)
    merged_path = ufc_repo_dir / 'data' / 'merged_stats_n_scorecards'
    
    if merged_path.exists():
        csv_files = list(merged_path.glob('*.csv'))
        if csv_files:
            print("📊 Loading merged UFC dataset...")
            # Load the first/largest CSV file
            main_file = csv_files[0]
            if len(csv_files) > 1:
                # Find the largest file (likely the main dataset)
                main_file = max(csv_files, key=lambda f: f.stat().st_size)
            
            try:
                ufc_df = pd.read_csv(main_file, nrows=1000)  # Preview first 1000 rows
                print(f"✓ Loaded: {main_file.name}")
                print(f"\nDataset shape (preview): {ufc_df.shape[0]} rows x {ufc_df.shape[1]} columns")
                print(f"\nColumns ({len(ufc_df.columns)} total):")
                for i, col in enumerate(ufc_df.columns, 1):
                    print(f"   {i}. {col}")
                
                print(f"\n📋 First few rows:")
                display(ufc_df.head())
                
                print(f"\n📈 Data types:")
                print(ufc_df.dtypes.value_counts())
                
            except Exception as e:
                print(f"⚠️ Error loading file: {e}")
        else:
            print("⚠️ No CSV files found in merged_stats_n_scorecards directory")
    else:
        # Try loading from stats directory
        stats_path = ufc_repo_dir / 'data' / 'stats'
        if stats_path.exists():
            csv_files = list(stats_path.glob('*.csv'))
            if csv_files:
                print(f"📊 Found {len(csv_files)} CSV files in stats directory")
                print("Loading first file as preview...")
                try:
                    ufc_df = pd.read_csv(csv_files[0], nrows=100)
                    print(f"✓ Loaded: {csv_files[0].name}")
                    print(f"Shape: {ufc_df.shape}")
                    display(ufc_df.head())
                except Exception as e:
                    print(f"⚠️ Error: {e}")
            else:
                print("⚠️ No CSV files found in stats directory")
        else:
            print("⚠️ Could not find data directories. Checking repository structure...")
            # List what's actually in the repo
            for item in ufc_repo_dir.iterdir():
                if item.is_dir():
                    print(f"   Directory: {item.name}")
else:
    print("⚠️ Repository not found. Please clone it first using the previous cell.")


In [2]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import numpy as np
import random


## 1. Generate Sample Viewing Logs Data

**Note:** In production, this section would load real viewing logs from your data source. 
The sample data generator below creates realistic patterns that can be easily replaced with actual data loading.


In [3]:
def generate_viewing_logs(num_users=5000, days=30, seed=42):
    """
    Generate realistic viewing logs for Paramount+ streaming data.
    
    Parameters:
    - num_users: Number of unique users
    - days: Number of days to generate data for
    - seed: Random seed for reproducibility
    
    Returns:
    - DataFrame with viewing logs
    """
    np.random.seed(seed)
    random.seed(seed)
    
    # Content IDs for UFC, UEFA Champions League, and NFL
    ufc_content_ids = [f"UFC_{i:03d}" for i in range(1, 21)]
    uefa_content_ids = [f"UEFA_{i:03d}" for i in range(1, 31)]
    nfl_content_ids = [f"NFL_{i:03d}" for i in range(1, 41)]
    all_content_ids = ufc_content_ids + uefa_content_ids + nfl_content_ids
    
    device_types = ['mobile', 'TV', 'web', 'tablet']
    actions = ['play', 'pause', 'rewind', 'replay', 'stop']
    
    logs = []
    start_date = datetime.now() - timedelta(days=days)
    
    # Generate sessions for each user
    for user_id in range(1, num_users + 1):
        num_sessions = np.random.poisson(lam=3)  # Average 3 sessions per user
        
        for session_num in range(num_sessions):
            session_id = f"SESSION_{user_id}_{session_num}"
            
            # Session start time
            session_start = start_date + timedelta(
                days=np.random.randint(0, days),
                hours=np.random.randint(0, 24),
                minutes=np.random.randint(0, 60)
            )
            
            # User signs up or starts browsing
            signup_time = session_start
            
            # Some users drop off before selecting content
            if np.random.random() < 0.15:  # 15% drop-off at signup
                logs.append({
                    'user_id': f"USER_{user_id:05d}",
                    'session_id': session_id,
                    'content_id': None,
                    'timestamp': signup_time,
                    'action': 'signup',
                    'device_type': random.choice(device_types),
                    'watch_duration_seconds': 0
                })
                continue
            
            # Select content
            content_id = random.choice(all_content_ids)
            content_start = session_start + timedelta(seconds=np.random.randint(0, 300))
            
            # Some users drop off during stream start
            if np.random.random() < 0.10:  # 10% drop-off at stream start
                logs.append({
                    'user_id': f"USER_{user_id:05d}",
                    'session_id': session_id,
                    'content_id': content_id,
                    'timestamp': content_start,
                    'action': 'play',
                    'device_type': random.choice(device_types),
                    'watch_duration_seconds': np.random.randint(0, 30)
                })
                continue
            
            # Generate watch events
            current_time = content_start
            total_watch_time = 0
            max_watch_time = np.random.exponential(scale=1800)  # Average 30 minutes
            
            while total_watch_time < max_watch_time and (current_time - content_start).total_seconds() < 7200:  # Max 2 hours
                action = random.choices(
                    actions,
                    weights=[50, 10, 5, 3, 2],  # play most common, stop least
                    k=1
                )[0]
                
                if action == 'play':
                    duration = np.random.exponential(scale=300)  # Average 5 minutes
                    total_watch_time += duration
                    logs.append({
                        'user_id': f"USER_{user_id:05d}",
                        'session_id': session_id,
                        'content_id': content_id,
                        'timestamp': current_time,
                        'action': 'play',
                        'device_type': random.choice(device_types),
                        'watch_duration_seconds': duration
                    })
                    current_time += timedelta(seconds=duration)
                elif action == 'pause':
                    pause_duration = np.random.randint(10, 300)
                    logs.append({
                        'user_id': f"USER_{user_id:05d}",
                        'session_id': session_id,
                        'content_id': content_id,
                        'timestamp': current_time,
                        'action': 'pause',
                        'device_type': random.choice(device_types),
                        'watch_duration_seconds': 0
                    })
                    current_time += timedelta(seconds=pause_duration)
                elif action == 'rewind':
                    rewind_seconds = np.random.randint(5, 60)
                    logs.append({
                        'user_id': f"USER_{user_id:05d}",
                        'session_id': session_id,
                        'content_id': content_id,
                        'timestamp': current_time,
                        'action': 'rewind',
                        'device_type': random.choice(device_types),
                        'watch_duration_seconds': 0
                    })
                    current_time += timedelta(seconds=5)  # Small time increment
                elif action == 'replay':
                    logs.append({
                        'user_id': f"USER_{user_id:05d}",
                        'session_id': session_id,
                        'content_id': content_id,
                        'timestamp': current_time,
                        'action': 'replay',
                        'device_type': random.choice(device_types),
                        'watch_duration_seconds': 0
                    })
                    current_time += timedelta(seconds=10)
                elif action == 'stop':
                    logs.append({
                        'user_id': f"USER_{user_id:05d}",
                        'session_id': session_id,
                        'content_id': content_id,
                        'timestamp': current_time,
                        'action': 'stop',
                        'device_type': random.choice(device_types),
                        'watch_duration_seconds': 0
                    })
                    break
    
    df = pd.DataFrame(logs)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df.sort_values('timestamp').reset_index(drop=True)

# Generate viewing logs
# TODO: Replace this with actual data loading: viewing_logs_df = pd.read_csv('viewing_logs.csv')
# Reduced number of users for faster execution - increase if needed
print("Generating viewing logs... This may take a moment...")
viewing_logs_df = generate_viewing_logs(num_users=2000, days=30)
print(f"Generated {len(viewing_logs_df):,} viewing log entries")
print(f"Date range: {viewing_logs_df['timestamp'].min()} to {viewing_logs_df['timestamp'].max()}")
viewing_logs_df.head()


Generating viewing logs... This may take a moment...
Generated 34,957 viewing log entries
Date range: 2025-11-02 14:05:58.172217 to 2025-12-02 14:56:07.187522


,user_id,session_id,content_id,timestamp,action,device_type,watch_duration_seconds
0,USER_00844,SESSION_844_3,None,2025-11-02 14:05:58.172217,signup,web,0.000000
1,USER_00404,SESSION_404_0,NFL_021,2025-11-02 14:09:10.172217,pause,TV,0.000000
2,USER_00404,SESSION_404_0,NFL_021,2025-11-02 14:09:36.172217,play,web,47.593430
3,USER_00404,SESSION_404_0,NFL_021,2025-11-02 14:10:23.765647,play,TV,551.513648
4,USER_00338,SESSION_338_2,NFL_014,2025-11-02 14:18:26.172217,play,mobile,1.830567


## 2. Generate Content Metadata

**Note:** In production, this section would load real content metadata from your database or API.


In [4]:
def generate_content_metadata():
    """
    Generate content metadata for UFC, UEFA Champions League, and NFL content.
    
    Returns:
    - DataFrame with content metadata
    """
    metadata = []
    
    # UFC Content
    ufc_events = [
        "UFC 285: Jones vs Gane", "UFC 286: Edwards vs Usman 3", "UFC 287: Pereira vs Adesanya 2",
        "UFC Fight Night: Vera vs Sandhagen", "UFC 288: Sterling vs Cejudo",
        "UFC Fight Night: Dern vs Hill", "UFC 289: Nunes vs Aldana", "UFC Fight Night: Vettori vs Cannonier",
        "UFC 290: Volkanovski vs Rodriguez", "UFC Fight Night: Aspinall vs Tybura",
        "UFC 291: Poirier vs Gaethje 2", "UFC Fight Night: Luque vs dos Anjos",
        "UFC 292: Sterling vs O'Malley", "UFC Fight Night: Gane vs Spivak",
        "UFC 293: Adesanya vs Strickland", "UFC Fight Night: Fiziev vs Gamrot",
        "UFC 294: Makhachev vs Oliveira 2", "UFC Fight Night: Allen vs Craig",
        "UFC 295: Prochazka vs Pereira", "UFC Fight Night: Blanchfield vs Fiorot"
    ]
    
    for i, event_name in enumerate(ufc_events, 1):
        content_id = f"UFC_{i:03d}"
        metadata.append({
            'content_id': content_id,
            'sport': 'UFC',
            'league': 'UFC',
            'team_1': None,
            'team_2': None,
            'language': 'English',
            'is_live': random.choice([True, False]),
            'tournament_stage': random.choice(['Main Card', 'Prelims', 'Early Prelims']),
            'match_event_name': event_name,
            'event_date': datetime.now() - timedelta(days=np.random.randint(1, 90))
        })
    
    # UEFA Champions League Content
    uefa_teams = [
        ('Manchester City', 'Real Madrid'), ('Bayern Munich', 'PSG'), ('Liverpool', 'Barcelona'),
        ('Chelsea', 'AC Milan'), ('Arsenal', 'Napoli'), ('Inter Milan', 'Atletico Madrid'),
        ('Borussia Dortmund', 'Sevilla'), ('Porto', 'Benfica'), ('Juventus', 'Roma'),
        ('Tottenham', 'Newcastle'), ('Real Madrid', 'Manchester City'), ('PSG', 'Bayern Munich'),
        ('Barcelona', 'Liverpool'), ('AC Milan', 'Chelsea'), ('Napoli', 'Arsenal'),
        ('Atletico Madrid', 'Inter Milan'), ('Sevilla', 'Borussia Dortmund'), ('Benfica', 'Porto'),
        ('Roma', 'Juventus'), ('Newcastle', 'Tottenham'), ('Manchester City', 'Bayern Munich'),
        ('Real Madrid', 'PSG'), ('Liverpool', 'AC Milan'), ('Barcelona', 'Napoli'),
        ('Chelsea', 'Inter Milan'), ('Arsenal', 'Atletico Madrid'), ('Borussia Dortmund', 'Porto'),
        ('Sevilla', 'Benfica'), ('Juventus', 'Tottenham'), ('Roma', 'Newcastle')
    ]
    
    stages = ['Group Stage', 'Round of 16', 'Quarter-Finals', 'Semi-Finals', 'Final']
    
    for i, (team1, team2) in enumerate(uefa_teams, 1):
        content_id = f"UEFA_{i:03d}"
        stage = random.choice(stages)
        metadata.append({
            'content_id': content_id,
            'sport': 'UEFA Champions League',
            'league': 'UEFA Champions League',
            'team_1': team1,
            'team_2': team2,
            'language': random.choice(['English', 'Spanish', 'French', 'German', 'Italian']),
            'is_live': random.choice([True, False]),
            'tournament_stage': stage,
            'match_event_name': f"{team1} vs {team2}",
            'event_date': datetime.now() - timedelta(days=np.random.randint(1, 120))
        })
    
    # NFL Content
    nfl_teams = [
        ('Kansas City Chiefs', 'Philadelphia Eagles'), ('Buffalo Bills', 'Cincinnati Bengals'),
        ('San Francisco 49ers', 'Dallas Cowboys'), ('Miami Dolphins', 'Baltimore Ravens'),
        ('Detroit Lions', 'Green Bay Packers'), ('Tampa Bay Buccaneers', 'New Orleans Saints'),
        ('New York Giants', 'Washington Commanders'), ('Los Angeles Rams', 'Seattle Seahawks'),
        ('Minnesota Vikings', 'Chicago Bears'), ('Arizona Cardinals', 'Los Angeles Chargers'),
        ('Denver Broncos', 'Las Vegas Raiders'), ('Jacksonville Jaguars', 'Tennessee Titans'),
        ('Indianapolis Colts', 'Houston Texans'), ('Cleveland Browns', 'Pittsburgh Steelers'),
        ('New York Jets', 'New England Patriots'), ('Carolina Panthers', 'Atlanta Falcons'),
        ('Philadelphia Eagles', 'Kansas City Chiefs'), ('Cincinnati Bengals', 'Buffalo Bills'),
        ('Dallas Cowboys', 'San Francisco 49ers'), ('Baltimore Ravens', 'Miami Dolphins'),
        ('Green Bay Packers', 'Detroit Lions'), ('New Orleans Saints', 'Tampa Bay Buccaneers'),
        ('Washington Commanders', 'New York Giants'), ('Seattle Seahawks', 'Los Angeles Rams'),
        ('Chicago Bears', 'Minnesota Vikings'), ('Los Angeles Chargers', 'Arizona Cardinals'),
        ('Las Vegas Raiders', 'Denver Broncos'), ('Tennessee Titans', 'Jacksonville Jaguars'),
        ('Houston Texans', 'Indianapolis Colts'), ('Pittsburgh Steelers', 'Cleveland Browns'),
        ('New England Patriots', 'New York Jets'), ('Atlanta Falcons', 'Carolina Panthers'),
        ('Kansas City Chiefs', 'Buffalo Bills'), ('San Francisco 49ers', 'Miami Dolphins'),
        ('Philadelphia Eagles', 'Dallas Cowboys'), ('Cincinnati Bengals', 'Baltimore Ravens'),
        ('Detroit Lions', 'Tampa Bay Buccaneers'), ('Green Bay Packers', 'New York Giants'),
        ('Los Angeles Rams', 'Minnesota Vikings'), ('Seattle Seahawks', 'Arizona Cardinals'),
        ('Denver Broncos', 'Jacksonville Jaguars')
    ]
    
    nfl_stages = ['Regular Season', 'Wild Card', 'Divisional Round', 'Conference Championship', 'Super Bowl']
    
    for i, (team1, team2) in enumerate(nfl_teams, 1):
        content_id = f"NFL_{i:03d}"
        stage = random.choice(nfl_stages)
        metadata.append({
            'content_id': content_id,
            'sport': 'NFL',
            'league': 'NFL',
            'team_1': team1,
            'team_2': team2,
            'language': 'English',
            'is_live': random.choice([True, False]),
            'tournament_stage': stage,
            'match_event_name': f"{team1} vs {team2}",
            'event_date': datetime.now() - timedelta(days=np.random.randint(1, 150))
        })
    
    df = pd.DataFrame(metadata)
    df['event_date'] = pd.to_datetime(df['event_date'])
    return df

# Generate content metadata
# TODO: Replace this with actual data loading: content_metadata_df = pd.read_csv('content_metadata.csv')
content_metadata_df = generate_content_metadata()
print(f"Generated metadata for {len(content_metadata_df)} content items")
print(f"\nBreakdown by sport:")
print(content_metadata_df['sport'].value_counts())
content_metadata_df.head(10)


Generated metadata for 91 content items

Breakdown by sport:
sport
NFL                      41
UEFA Champions League    30
UFC                      20
Name: count, dtype: int64


,content_id,sport,league,team_1,team_2,language,is_live,tournament_stage,match_event_name,event_date
0,UFC_001,UFC,UFC,None,None,English,False,Prelims,UFC 285: Jones vs Gane,2025-10-09 14:00:58.394602
1,UFC_002,UFC,UFC,None,None,English,False,Prelims,UFC 286: Edwards vs Usman 3,2025-10-19 14:00:58.394644
2,UFC_003,UFC,UFC,None,None,English,True,Early Prelims,UFC 287: Pereira vs Adesanya 2,2025-10-15 14:00:58.394649
3,UFC_004,UFC,UFC,None,None,English,False,Main Card,UFC Fight Night: Vera vs Sandhagen,2025-10-26 14:00:58.394652
4,UFC_005,UFC,UFC,None,None,English,True,Main Card,UFC 288: Sterling vs Cejudo,2025-10-07 14:00:58.394655
5,UFC_006,UFC,UFC,None,None,English,True,Early Prelims,UFC Fight Night: Dern vs Hill,2025-10-30 14:00:58.394658
6,UFC_007,UFC,UFC,None,None,English,False,Early Prelims,UFC 289: Nunes vs Aldana,2025-10-28 14:00:58.394661
7,UFC_008,UFC,UFC,None,None,English,False,Early Prelims,UFC Fight Night: Vettori vs Cannonier,2025-11-26 14:00:58.394667
8,UFC_009,UFC,UFC,None,None,English,False,Prelims,UFC 290: Volkanovski vs Rodriguez,2025-10-17 14:00:58.394671
9,UFC_010,UFC,UFC,None,None,English,True,Early Prelims,UFC Fight Night: Aspinall vs Tybura,2025-09-19 14:00:58.394674


## 3. Data Joining and Preparation


In [5]:
# Join viewing logs with content metadata
# Filter out rows where content_id is None (signup drop-offs)
viewing_data = viewing_logs_df[viewing_logs_df['content_id'].notna()].copy()

# Join with metadata
viewing_data = viewing_data.merge(
    content_metadata_df,
    on='content_id',
    how='left'
)

# Add calculated fields
viewing_data['watch_duration_minutes'] = viewing_data['watch_duration_seconds'] / 60
viewing_data['hour_of_day'] = viewing_data['timestamp'].dt.hour
viewing_data['day_of_week'] = viewing_data['timestamp'].dt.day_name()
viewing_data['date'] = viewing_data['timestamp'].dt.date

# Calculate session completion (simplified: if watch duration > 10 minutes, consider completed)
viewing_data['session_completed'] = (
    viewing_data.groupby('session_id')['watch_duration_seconds']
    .transform('sum') > 600
)

print(f"Total viewing records: {len(viewing_data):,}")
print(f"Unique users: {viewing_data['user_id'].nunique():,}")
print(f"Unique sessions: {viewing_data['session_id'].nunique():,}")
print(f"Unique content items: {viewing_data['content_id'].nunique():,}")
viewing_data.head()


Total viewing records: 34,062
Unique users: 1,847
Unique sessions: 5,046
Unique content items: 90


,user_id,session_id,content_id,timestamp,action,device_type,watch_duration_seconds,sport,league,team_1,...,language,is_live,tournament_stage,match_event_name,event_date,watch_duration_minutes,hour_of_day,day_of_week,date,session_completed
0,USER_00404,SESSION_404_0,NFL_021,2025-11-02 14:09:10.172217,pause,TV,0.000000,NFL,NFL,Green Bay Packers,...,English,True,Conference Championship,Green Bay Packers vs Detroit Lions,2025-11-03 14:00:58.394851,0.000000,14,Sunday,2025-11-02,True
1,USER_00404,SESSION_404_0,NFL_021,2025-11-02 14:09:36.172217,play,web,47.593430,NFL,NFL,Green Bay Packers,...,English,True,Conference Championship,Green Bay Packers vs Detroit Lions,2025-11-03 14:00:58.394851,0.793224,14,Sunday,2025-11-02,True
2,USER_00404,SESSION_404_0,NFL_021,2025-11-02 14:10:23.765647,play,TV,551.513648,NFL,NFL,Green Bay Packers,...,English,True,Conference Championship,Green Bay Packers vs Detroit Lions,2025-11-03 14:00:58.394851,9.191894,14,Sunday,2025-11-02,True
3,USER_00338,SESSION_338_2,NFL_014,2025-11-02 14:18:26.172217,play,mobile,1.830567,NFL,NFL,Cleveland Browns,...,English,False,Wild Card,Cleveland Browns vs Pittsburgh Steelers,2025-08-08 14:00:58.394830,0.030509,14,Sunday,2025-11-02,True
4,USER_00338,SESSION_338_2,NFL_014,2025-11-02 14:18:28.002784,play,tablet,191.202732,NFL,NFL,Cleveland Browns,...,English,False,Wild Card,Cleveland Browns vs Pittsburgh Steelers,2025-08-08 14:00:58.394830,3.186712,14,Sunday,2025-11-02,True


## 4. Build Metrics

### 4.1 Drop-off Analysis


In [6]:
# Calculate drop-off metrics
# Track users through the funnel: signup -> stream start -> completion

# Get all unique users who attempted to use the service
all_users = viewing_logs_df['user_id'].unique()

# Users who signed up (have signup action)
signup_events = viewing_logs_df[viewing_logs_df['action'] == 'signup']
users_with_signup = signup_events['user_id'].unique()

# Users who started streaming (have play action with content)
stream_start_events = viewing_logs_df[
    (viewing_logs_df['action'] == 'play') & 
    (viewing_logs_df['content_id'].notna())
]
users_started_stream = stream_start_events['user_id'].unique()

# Users who dropped off during stream start (played but watched < 30 seconds)
dropped_at_stream_start = viewing_logs_df[
    (viewing_logs_df['action'] == 'play') &
    (viewing_logs_df['watch_duration_seconds'] < 30) &
    (viewing_logs_df['content_id'].notna())
]['user_id'].unique()

# Calculate funnel stages properly
# Stage 1: All users who signed up
users_signed_up = len(users_with_signup)

# Stage 2: Users who signed up AND started streaming
users_signed_up_and_streamed = len(set(users_with_signup) & set(users_started_stream))

# Stage 3: Users who completed stream start (didn't drop off immediately)
users_completed_stream_start = users_signed_up_and_streamed - len(set(users_with_signup) & set(dropped_at_stream_start))

# Calculate conversion rates
signup_to_stream_rate = (users_signed_up_and_streamed / users_signed_up * 100) if users_signed_up > 0 else 0
stream_start_completion_rate = (users_completed_stream_start / users_signed_up_and_streamed * 100) if users_signed_up_and_streamed > 0 else 0

print("Drop-off Analysis")
print("=" * 50)
print(f"Users who signed up: {users_signed_up:,}")
print(f"Users who signed up AND started streaming: {users_signed_up_and_streamed:,}")
print(f"Users who completed stream start: {users_completed_stream_start:,}")
print(f"Users who dropped off at stream start: {users_signed_up_and_streamed - users_completed_stream_start:,}")
print(f"\nSignup to Stream Start Conversion Rate: {signup_to_stream_rate:.2f}%")
print(f"Stream Start Completion Rate: {stream_start_completion_rate:.2f}%")

# Create funnel visualization
# Funnel values must be in descending order
funnel_stages = ['Signed Up', 'Started Stream', 'Completed Stream Start']
funnel_values = [users_signed_up, users_signed_up_and_streamed, users_completed_stream_start]

# Only create funnel if we have valid data
if users_signed_up > 0 and all(v <= users_signed_up for v in funnel_values[1:]):
    fig = go.Figure(go.Funnel(
        y=funnel_stages,
        x=funnel_values,
        textposition="inside",
        textinfo="value+percent initial",
        marker={"color": ["#1f77b4", "#ff7f0e", "#2ca02c"]}
    ))
    
    fig.update_layout(
        title="User Drop-off Funnel: Signup to Stream Start",
        height=400,
        showlegend=False
    )
    
    # Display the funnel chart
    fig.show()
else:
    print("\n⚠️ Cannot create funnel chart: Invalid funnel data (values must decrease)")


Drop-off Analysis
Users who signed up: 712
Users who signed up AND started streaming: 651
Users who completed stream start: 196
Users who dropped off at stream start: 455

Signup to Stream Start Conversion Rate: 91.43%
Stream Start Completion Rate: 30.11%


### 4.2 Minutes Watched Analytics


In [7]:
# Aggregate watch time by sport
# Filter out any rows with missing sport data
viewing_data_clean = viewing_data[viewing_data['sport'].notna()].copy()

if len(viewing_data_clean) > 0:
    watch_time_by_sport = viewing_data_clean.groupby('sport')['watch_duration_minutes'].sum().reset_index()
    watch_time_by_sport.columns = ['Sport', 'Total Minutes Watched']
    watch_time_by_sport = watch_time_by_sport.sort_values('Total Minutes Watched', ascending=False)
    
    print("Total Minutes Watched by Sport")
    print("=" * 50)
    print(watch_time_by_sport.to_string(index=False))
    
    # Visualization
    if len(watch_time_by_sport) > 0:
        # Create color map dynamically based on available sports
        color_map = {
            'UFC': '#FF6B35',
            'UEFA Champions League': '#004E89',
            'NFL': '#1A659E'
        }
        
        # Only include colors for sports that exist in the data
        available_colors = {sport: color_map.get(sport, '#808080') for sport in watch_time_by_sport['Sport'].unique()}
        
        fig = px.bar(
            watch_time_by_sport,
            x='Sport',
            y='Total Minutes Watched',
            title='Total Minutes Watched by Sport',
            labels={'Total Minutes Watched': 'Total Minutes Watched', 'Sport': 'Sport'},
            color='Sport',
            color_discrete_map=available_colors
        )
        fig.update_yaxes(tickformat=',.0f')
        fig.update_layout(
            xaxis_title='Sport',
            yaxis_title='Total Minutes Watched',
            height=500
        )
        fig.show()
    else:
        print("\n⚠️ No data available for visualization")
else:
    print("⚠️ No viewing data available with sport information")


Total Minutes Watched by Sport
                Sport  Total Minutes Watched
                  NFL           52954.119093
UEFA Champions League           38562.431320
                  UFC           28840.719808


In [8]:
# Minutes watched by league (same as sport in this case, but structured for extensibility)
watch_time_by_league = viewing_data.groupby('league')['watch_duration_minutes'].sum().reset_index()
watch_time_by_league.columns = ['League', 'Total Minutes Watched']
watch_time_by_league = watch_time_by_league.sort_values('Total Minutes Watched', ascending=False)

fig = px.pie(
    watch_time_by_league,
    values='Total Minutes Watched',
    names='League',
    title='Watch Time Distribution by League',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.show()


In [9]:
# Minutes watched by team (for team sports)
team_watch_time = []

# UEFA Champions League teams
uefa_data = viewing_data[viewing_data['sport'] == 'UEFA Champions League']
for _, row in uefa_data.iterrows():
    if pd.notna(row['team_1']):
        team_watch_time.append({'Team': row['team_1'], 'Minutes': row['watch_duration_minutes']})
    if pd.notna(row['team_2']):
        team_watch_time.append({'Team': row['team_2'], 'Minutes': row['watch_duration_minutes']})

# NFL teams
nfl_data = viewing_data[viewing_data['sport'] == 'NFL']
for _, row in nfl_data.iterrows():
    if pd.notna(row['team_1']):
        team_watch_time.append({'Team': row['team_1'], 'Minutes': row['watch_duration_minutes']})
    if pd.notna(row['team_2']):
        team_watch_time.append({'Team': row['team_2'], 'Minutes': row['watch_duration_minutes']})

if team_watch_time:
    team_df = pd.DataFrame(team_watch_time)
    team_totals = team_df.groupby('Team')['Minutes'].sum().reset_index()
    team_totals = team_totals.sort_values('Minutes', ascending=False).head(20)
    
    fig = px.bar(
        team_totals,
        x='Minutes',
        y='Team',
        orientation='h',
        title='Top 20 Teams by Total Minutes Watched',
        labels={'Minutes': 'Total Minutes Watched'}
    )
    fig.update_layout(height=600)
    fig.show()


In [10]:
# Minutes watched: Live vs Replay
live_vs_replay = viewing_data.groupby('is_live')['watch_duration_minutes'].sum().reset_index()
live_vs_replay['Type'] = live_vs_replay['is_live'].map({True: 'Live', False: 'Replay'})
live_vs_replay.columns = ['is_live', 'Total Minutes Watched', 'Type']

print("Minutes Watched: Live vs Replay")
print("=" * 50)
print(live_vs_replay[['Type', 'Total Minutes Watched']].to_string(index=False))

fig = px.bar(
    live_vs_replay,
    x='Type',
    y='Total Minutes Watched',
    title='Watch Time: Live vs Replay',
    color='Type',
    color_discrete_map={'Live': '#FF6B35', 'Replay': '#004E89'}
)
fig.update_yaxes(tickformat=',.0f')
fig.show()


Minutes Watched: Live vs Replay
  Type  Total Minutes Watched
Replay           56971.622068
  Live           63385.648154


In [11]:
# Minutes watched by tournament stage
stage_watch_time = viewing_data.groupby('tournament_stage')['watch_duration_minutes'].sum().reset_index()
stage_watch_time.columns = ['Tournament Stage', 'Total Minutes Watched']
stage_watch_time = stage_watch_time.sort_values('Total Minutes Watched', ascending=False)

print("Minutes Watched by Tournament Stage")
print("=" * 50)
print(stage_watch_time.to_string(index=False))

fig = px.bar(
    stage_watch_time,
    x='Tournament Stage',
    y='Total Minutes Watched',
    title='Watch Time by Tournament Stage',
    color='Total Minutes Watched',
    color_continuous_scale='Viridis'
)
fig.update_xaxes(tickangle=45)
fig.update_yaxes(tickformat=',.0f')
fig.show()


Minutes Watched by Tournament Stage
       Tournament Stage  Total Minutes Watched
              Wild Card           16884.901716
Conference Championship           12803.191131
          Early Prelims           11246.392670
             Super Bowl            9435.770417
                Prelims            9003.980871
            Group Stage            8998.728551
              Main Card            8590.346268
                  Final            8476.930022
         Regular Season            8252.165240
         Quarter-Finals            8173.489158
            Semi-Finals            7459.589678
       Divisional Round            5578.090588
            Round of 16            5453.693912


### 4.3 Big Moments Detection


In [12]:
# Calculate concurrent viewers over time
# Group by timestamp (rounded to nearest minute) and content
viewing_data['timestamp_minute'] = viewing_data['timestamp'].dt.floor('min')

concurrent_viewers = viewing_data.groupby(['timestamp_minute', 'content_id']).agg({
    'user_id': 'nunique',
    'sport': 'first',
    'match_event_name': 'first'
}).reset_index()
concurrent_viewers.columns = ['timestamp_minute', 'content_id', 'concurrent_viewers', 'sport', 'match_event_name']

# Find spikes (moments with unusually high concurrent viewers)
# Use z-score to identify outliers
mean_viewers = concurrent_viewers['concurrent_viewers'].mean()
std_viewers = concurrent_viewers['concurrent_viewers'].std()

if std_viewers > 0:
    concurrent_viewers['z_score'] = (concurrent_viewers['concurrent_viewers'] - mean_viewers) / std_viewers
    # Big moments: z-score > 2 (top ~2.5% of moments)
    big_moments = concurrent_viewers[concurrent_viewers['z_score'] > 2].copy()
    big_moments = big_moments.sort_values('concurrent_viewers', ascending=False)
else:
    # If std is 0, all values are the same, so use top 10% by concurrent viewers
    threshold = concurrent_viewers['concurrent_viewers'].quantile(0.9)
    big_moments = concurrent_viewers[concurrent_viewers['concurrent_viewers'] >= threshold].copy()
    big_moments = big_moments.sort_values('concurrent_viewers', ascending=False)

print(f"Found {len(big_moments)} big moments (spikes in concurrent viewers)")
print("\nTop 10 Big Moments:")
print("=" * 80)
if len(big_moments) > 0:
    print(big_moments[['timestamp_minute', 'match_event_name', 'sport', 'concurrent_viewers']].head(10).to_string(index=False))
else:
    print("No big moments detected")


Found 106 big moments (spikes in concurrent viewers)

Top 10 Big Moments:
   timestamp_minute                         match_event_name                 sport  concurrent_viewers
2025-11-03 03:37:00           UFC 287: Pereira vs Adesanya 2                   UFC                   2
2025-11-24 20:17:00            UFC 291: Poirier vs Gaethje 2                   UFC                   2
2025-11-24 07:00:00 Washington Commanders vs New York Giants                   NFL                   2
2025-11-24 06:27:00           Inter Milan vs Atletico Madrid UEFA Champions League                   2
2025-11-24 06:06:00                   Newcastle vs Tottenham UEFA Champions League                   2
2025-11-24 04:26:00                      Chelsea vs AC Milan UEFA Champions League                   2
2025-11-24 04:22:00                      Chelsea vs AC Milan UEFA Champions League                   2
2025-11-23 21:22:00   UFC Fight Night: Blanchfield vs Fiorot                   UFC                   2

In [13]:
# Detect rewind/replay patterns (indicates key moments)
rewind_replay_data = viewing_data[viewing_data['action'].isin(['rewind', 'replay'])].copy()

if len(rewind_replay_data) > 0:
    # Group by content and timestamp (rounded to 5-minute intervals)
    rewind_replay_data['timestamp_5min'] = rewind_replay_data['timestamp'].dt.floor('5min')
    
    rewind_replay_counts = rewind_replay_data.groupby(['timestamp_5min', 'content_id']).agg({
        'action': 'count',
        'sport': 'first',
        'match_event_name': 'first'
    }).reset_index()
    rewind_replay_counts.columns = ['timestamp_5min', 'content_id', 'rewind_replay_count', 'sport', 'match_event_name']
    
    # Find moments with high rewind/replay activity
    mean_rewind = rewind_replay_counts['rewind_replay_count'].mean()
    std_rewind = rewind_replay_counts['rewind_replay_count'].std()
    
    if std_rewind > 0:
        rewind_replay_counts['z_score'] = (rewind_replay_counts['rewind_replay_count'] - mean_rewind) / std_rewind
        high_rewind_moments = rewind_replay_counts[rewind_replay_counts['z_score'] > 2].copy()
    else:
        # If std is 0, use top 10% by count
        threshold = rewind_replay_counts['rewind_replay_count'].quantile(0.9)
        high_rewind_moments = rewind_replay_counts[rewind_replay_counts['rewind_replay_count'] >= threshold].copy()
    
    high_rewind_moments = high_rewind_moments.sort_values('rewind_replay_count', ascending=False)
    
    print(f"Found {len(high_rewind_moments)} moments with high rewind/replay activity")
    print("\nTop 10 Moments with Most Rewinds/Replays:")
    print("=" * 80)
    if len(high_rewind_moments) > 0:
        print(high_rewind_moments[['timestamp_5min', 'match_event_name', 'sport', 'rewind_replay_count']].head(10).to_string(index=False))
    else:
        print("No high rewind/replay moments detected")
else:
    high_rewind_moments = pd.DataFrame(columns=['timestamp_5min', 'match_event_name', 'sport', 'rewind_replay_count'])
    print("No rewind/replay data found")


Found 90 moments with high rewind/replay activity

Top 10 Moments with Most Rewinds/Replays:
     timestamp_5min                          match_event_name                 sport  rewind_replay_count
2025-11-25 18:45:00           UFC 293: Adesanya vs Strickland                   UFC                    4
2025-11-04 22:40:00                        Real Madrid vs PSG UEFA Champions League                    4
2025-11-07 00:00:00   Pittsburgh Steelers vs Cleveland Browns                   NFL                    4
2025-11-27 13:25:00           UFC Fight Night: Allen vs Craig                   UFC                    4
2025-11-05 19:55:00        Minnesota Vikings vs Chicago Bears                   NFL                    4
2025-11-05 11:15:00           UFC Fight Night: Allen vs Craig                   UFC                    4
2025-11-27 02:00:00   Cleveland Browns vs Pittsburgh Steelers                   NFL                    4
2025-11-05 08:40:00 Philadelphia Eagles vs Kansas City Chiefs      

In [14]:
# Visualize concurrent viewers over time for top content
top_content = concurrent_viewers.groupby('content_id')['concurrent_viewers'].max().nlargest(5).index

fig = go.Figure()

for content_id in top_content:
    content_data = concurrent_viewers[concurrent_viewers['content_id'] == content_id].sort_values('timestamp_minute')
    match_name = content_data['match_event_name'].iloc[0]
    sport = content_data['sport'].iloc[0]
    
    fig.add_trace(go.Scatter(
        x=content_data['timestamp_minute'],
        y=content_data['concurrent_viewers'],
        mode='lines',
        name=f"{sport}: {match_name[:30]}...",
        line=dict(width=2)
    ))

fig.update_layout(
    title='Concurrent Viewers Over Time - Top 5 Content Items',
    xaxis_title='Time',
    yaxis_title='Concurrent Viewers',
    height=500,
    hovermode='x unified'
)
fig.show()


## 5. Moment Map Visualizations

Create visualizations showing where fandom is intense, where people churn, and where they binge.


In [15]:
# Heatmap: Engagement intensity by hour of day and day of week
engagement_by_time = viewing_data.groupby(['day_of_week', 'hour_of_day'])['watch_duration_minutes'].sum().reset_index()

# Create pivot table for heatmap
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
engagement_by_time['day_of_week'] = pd.Categorical(engagement_by_time['day_of_week'], categories=day_order, ordered=True)
engagement_pivot = engagement_by_time.pivot(index='day_of_week', columns='hour_of_day', values='watch_duration_minutes')

fig = px.imshow(
    engagement_pivot,
    labels=dict(x="Hour of Day", y="Day of Week", color="Minutes Watched"),
    title="Engagement Heatmap: Watch Time by Day and Hour",
    color_continuous_scale='YlOrRd',
    aspect="auto"
)
fig.update_layout(height=400)
fig.show()


In [16]:
# Churn points visualization: Show where users drop off
churn_analysis = viewing_data.groupby('content_id').agg({
    'user_id': 'nunique',
    'watch_duration_minutes': 'sum',
    'sport': 'first',
    'match_event_name': 'first'
}).reset_index()

# Calculate average watch duration per user per content
churn_analysis['avg_watch_per_user'] = churn_analysis['watch_duration_minutes'] / churn_analysis['user_id']
churn_analysis['churn_score'] = 1 / (1 + churn_analysis['avg_watch_per_user'])  # Higher score = more churn

# Top content by churn (low engagement relative to viewers)
churn_analysis = churn_analysis.sort_values('churn_score', ascending=False).head(20)

fig = px.scatter(
    churn_analysis,
    x='user_id',
    y='avg_watch_per_user',
    size='watch_duration_minutes',
    color='sport',
    hover_data=['match_event_name'],
    title='Churn Analysis: Content with High Viewer Drop-off',
    labels={
        'user_id': 'Number of Viewers',
        'avg_watch_per_user': 'Average Minutes Watched per User',
        'sport': 'Sport'
    },
    color_discrete_map={
        'UFC': '#FF6B35',
        'UEFA Champions League': '#004E89',
        'NFL': '#1A659E'
    }
)
fig.update_layout(height=500)
fig.show()


In [17]:
# Binge-watching patterns: Identify users who watch multiple sessions/content in short time
# Calculate sessions per user per day
user_daily_sessions = viewing_data.groupby(['user_id', 'date']).agg({
    'session_id': 'nunique',
    'watch_duration_minutes': 'sum',
    'content_id': 'nunique'
}).reset_index()
user_daily_sessions.columns = ['user_id', 'date', 'sessions_per_day', 'total_minutes', 'unique_content']

# Identify binge watchers: >3 sessions in a day or >2 hours watched in a day
binge_watchers = user_daily_sessions[
    (user_daily_sessions['sessions_per_day'] >= 3) | 
    (user_daily_sessions['total_minutes'] >= 120)
].copy()

print(f"Found {binge_watchers['user_id'].nunique()} binge watchers")
print(f"Total binge-watching days: {len(binge_watchers)}")

# Visualize binge patterns by sport
binge_by_sport = viewing_data[viewing_data['user_id'].isin(binge_watchers['user_id'])].groupby('sport')['watch_duration_minutes'].sum().reset_index()
binge_by_sport.columns = ['Sport', 'Binge Watch Minutes']

fig = px.bar(
    binge_by_sport,
    x='Sport',
    y='Binge Watch Minutes',
    title='Binge-Watching Patterns by Sport',
    color='Sport',
    color_discrete_map={
        'UFC': '#FF6B35',
        'UEFA Champions League': '#004E89',
        'NFL': '#1A659E'
    }
)
fig.update_yaxes(tickformat=',.0f')
fig.show()


Found 20 binge watchers
Total binge-watching days: 20


In [18]:
# Device-based engagement patterns
device_engagement = viewing_data.groupby(['device_type', 'sport']).agg({
    'watch_duration_minutes': 'sum',
    'user_id': 'nunique'
}).reset_index()
device_engagement.columns = ['Device Type', 'Sport', 'Total Minutes', 'Unique Users']

fig = px.bar(
    device_engagement,
    x='Device Type',
    y='Total Minutes',
    color='Sport',
    title='Watch Time by Device Type and Sport',
    barmode='group',
    color_discrete_map={
        'UFC': '#FF6B35',
        'UEFA Champions League': '#004E89',
        'NFL': '#1A659E'
    }
)
fig.update_yaxes(tickformat=',.0f')
fig.show()


In [19]:
# Engagement intensity map: Content with highest engagement (combination of viewers, watch time, and rewinds)
content_engagement = viewing_data.groupby('content_id').agg({
    'user_id': 'nunique',
    'watch_duration_minutes': 'sum',
    'sport': 'first',
    'match_event_name': 'first',
    'tournament_stage': 'first'
}).reset_index()

# Count rewinds/replays per content
rewind_counts = viewing_data[viewing_data['action'].isin(['rewind', 'replay'])].groupby('content_id').size().reset_index()
rewind_counts.columns = ['content_id', 'rewind_replay_count']

content_engagement = content_engagement.merge(rewind_counts, on='content_id', how='left')
content_engagement['rewind_replay_count'] = content_engagement['rewind_replay_count'].fillna(0)

# Calculate engagement score (normalized combination of metrics)
# Handle division by zero cases
max_users = content_engagement['user_id'].max()
max_minutes = content_engagement['watch_duration_minutes'].max()
max_rewinds = content_engagement['rewind_replay_count'].max()

if max_users > 0 and max_minutes > 0:
    if max_rewinds > 0:
        content_engagement['engagement_score'] = (
            (content_engagement['user_id'] / max_users) * 0.4 +
            (content_engagement['watch_duration_minutes'] / max_minutes) * 0.4 +
            (content_engagement['rewind_replay_count'] / max_rewinds) * 0.2
        )
    else:
        # If no rewinds, use only users and minutes
        content_engagement['engagement_score'] = (
            (content_engagement['user_id'] / max_users) * 0.5 +
            (content_engagement['watch_duration_minutes'] / max_minutes) * 0.5
        )
    
    top_engagement = content_engagement.nlargest(15, 'engagement_score')
    
    fig = px.scatter(
        top_engagement,
        x='user_id',
        y='watch_duration_minutes',
        size='rewind_replay_count',
        color='sport',
        hover_data=['match_event_name', 'tournament_stage', 'engagement_score'],
        title='Top 15 Content by Engagement Intensity (Fandom Map)',
        labels={
            'user_id': 'Number of Viewers',
            'watch_duration_minutes': 'Total Minutes Watched',
            'rewind_replay_count': 'Rewinds/Replays',
            'sport': 'Sport'
        },
        color_discrete_map={
            'UFC': '#FF6B35',
            'UEFA Champions League': '#004E89',
            'NFL': '#1A659E'
        }
    )
    fig.update_layout(height=600)
    fig.show()
else:
    print("Insufficient data for engagement score calculation")


## 6. Key Insights and Actionable Recommendations

Based on the analysis above, here are key insights that can inspire new experiences:


In [20]:
# Summary statistics
print("=" * 80)
print("KEY INSIGHTS SUMMARY")
print("=" * 80)

try:
    print("\n1. DROP-OFF INSIGHTS:")
    print(f"   - {signup_to_stream_rate:.1f}% of users who sign up actually start streaming")
    print(f"   - {stream_start_completion_rate:.1f}% complete stream start (don't drop off immediately)")
    print("   → ACTION: Optimize signup flow and reduce friction at stream start")
except NameError:
    print("\n1. DROP-OFF INSIGHTS: Run drop-off analysis cell first")

try:
    print("\n2. ENGAGEMENT BY SPORT:")
    for _, row in watch_time_by_sport.iterrows():
        print(f"   - {row['Sport']}: {row['Total Minutes Watched']:,.0f} minutes watched")
    print("   → ACTION: Allocate marketing budget proportionally to high-engagement sports")
except NameError:
    print("\n2. ENGAGEMENT BY SPORT: Run minutes watched analysis cell first")

try:
    print("\n3. BIG MOMENTS:")
    print(f"   - Found {len(big_moments)} moments with spikes in concurrent viewers")
    if 'high_rewind_moments' in locals() or 'high_rewind_moments' in globals():
        print(f"   - Found {len(high_rewind_moments)} moments with high rewind/replay activity")
    print("   → ACTION: Create custom countdowns and FOMO banners for upcoming similar content")
except NameError:
    print("\n3. BIG MOMENTS: Run big moments detection cell first")

try:
    print("\n4. BINGE PATTERNS:")
    print(f"   - {binge_watchers['user_id'].nunique()} users identified as binge watchers")
    print("   → ACTION: Implement smarter reminders and personalized recommendations for binge watchers")
except NameError:
    print("\n4. BINGE PATTERNS: Run binge-watching analysis cell first")

print("\n5. CHURN POINTS:")
print("   - Content with high viewer count but low average watch time indicates churn")
print("   → ACTION: Analyze content quality and improve recommendations for these users")

try:
    print("\n6. DEVICE PREFERENCES:")
    device_totals = viewing_data.groupby('device_type')['watch_duration_minutes'].sum().sort_values(ascending=False)
    total_minutes = device_totals.sum()
    if total_minutes > 0:
        for device, minutes in device_totals.items():
            print(f"   - {device}: {minutes:,.0f} minutes ({minutes/total_minutes*100:.1f}%)")
    print("   → ACTION: Optimize experience for dominant device types")
except NameError:
    print("\n6. DEVICE PREFERENCES: Run device analysis cell first")

print("\n" + "=" * 80)


KEY INSIGHTS SUMMARY

1. DROP-OFF INSIGHTS:
   - 91.4% of users who sign up actually start streaming
   - 30.1% complete stream start (don't drop off immediately)
   → ACTION: Optimize signup flow and reduce friction at stream start

2. ENGAGEMENT BY SPORT:
   - NFL: 52,954 minutes watched
   - UEFA Champions League: 38,562 minutes watched
   - UFC: 28,841 minutes watched
   → ACTION: Allocate marketing budget proportionally to high-engagement sports

3. BIG MOMENTS:
   - Found 106 moments with spikes in concurrent viewers
   - Found 90 moments with high rewind/replay activity
   → ACTION: Create custom countdowns and FOMO banners for upcoming similar content

4. BINGE PATTERNS:
   - 20 users identified as binge watchers
   → ACTION: Implement smarter reminders and personalized recommendations for binge watchers

5. CHURN POINTS:
   - Content with high viewer count but low average watch time indicates churn
   → ACTION: Analyze content quality and improve recommendations for these user